## Introduction

The Variational Quantum Eigensolver (VQE) is a flagship algorithm for quantum chemistry
using near-term quantum computers. It is an application of the
[Ritz variational principle](https://en.wikipedia.org/wiki/Ritz_method>), where a quantum
computer is trained to prepare the ground state of a given molecule.

The inputs to the VQE algorithm are a molecular Hamiltonian and a
parametrized circuit preparing the quantum state of the molecule. Within VQE, the
cost function is defined as the expectation value of the Hamiltonian computed in the
trial state. The ground state of the target Hamiltonian is obtained by performing an
iterative minimization of the cost function. The optimization is carried out
by a classical optimizer which leverages a quantum computer to evaluate the cost function
and calculate its gradient at each optimization step.

In this tutorial you will learn how to implement the VQE algorithm in a few lines of code.
As an illustrative example, we use it to find the ground state of the hydrogen
molecule, :math:`\mathrm{H}_2`. First, we build the molecular Hamiltonian using a minimal
basis set approximation. Next, we design the quantum circuit preparing the trial
state of the molecule, and the cost function to evaluate the expectation value
of the Hamiltonian. Finally, we select a classical optimizer, initialize the
circuit parameters, and run the VQE algorithm using a PennyLane simulator.

[The code and examples here are taken from Pennylane](https://github.com/PennyLaneAI/qml/blob/master/demonstrations/tutorial_vqe.py)

## Let's get started!

### Building the electronic Hamiltonian

The first step is to specify the molecule we want to simulate. **aqora** provides you with this molecule in the form of an input. We mark the cell where the inputs are placed with the `parameters` tag

In [1]:
input = """2
Sample H2 molecule
H 0.3710 0.0 0.0
H -0.3710 0.0 0.0"""

In chemistry we often use the [XYZ File Format](https://en.wikipedia.org/wiki/XYZ_file_format) which describes the location of atoms in a molecule. To make use of it in the code, we need to to parse it first

In [2]:
import xyz_parse
molecule = xyz_parse.Molecule.parse(input)
repr(molecule)

'Molecule { comment: "Sample H2 molecule", atoms: [Atom { symbol: "H", x: 0.3710, y: 0.0, z: 0.0 }, Atom { symbol: "H", x: -0.3710, y: 0.0, z: 0.0 }] }'

Now, we can build the electronic Hamiltonian of the hydrogen molecule using the function `pennylane.qchem.molecular_hamiltonian` function.

In [3]:
import pennylane as qml
from pennylane import numpy as np

# Setup hamiltonian
H, qubits = qml.qchem.molecular_hamiltonian(
    molecule.symbols,
    # Factor of 1.88973 to convert Angstrom to Bohr
    np.array(molecule.coordinates, dtype=np.float64) * 1.88973
)
print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

Number of qubits =  4
The Hamiltonian is  -0.09963387941391993 * I(0) + 0.1711054512373633 * Z(0) + 0.17110545123736323 * Z(1) + 0.16859349595528597 * (Z(0) @ Z(1)) + -0.2225091423657487 * Z(2) + 0.1205102798953805 * (Z(0) @ Z(2)) + 0.16584090244108435 * (Z(1) @ Z(2)) + 0.045330622545703844 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.045330622545703844 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.045330622545703844 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 0.045330622545703844 * (X(0) @ Y(1) @ Y(2) @ X(3)) + -0.2225091423657487 * Z(3) + 0.16584090244108435 * (Z(0) @ Z(3)) + 0.1205102798953805 * (Z(1) @ Z(3)) + 0.17432077259222362 * (Z(2) @ Z(3))


The outputs of the function are the Hamiltonian, represented as
a linear combination of Pauli operators, and the number of qubits
required for the quantum simulations. For this example, we use a
[minimal basis set](<https://en.wikipedia.org/wiki/STO-nG_basis_sets>)
to represent the [molecular orbitals](https://en.wikipedia.org/wiki/Molecular_orbital>).
In this approximation, we have four spin orbitals, which defines the
number of qubits.

#### Implementing the VQE algorithm

From here on, we can use PennyLane as usual, employing its entire stack of
algorithms and optimizers. We begin by defining the device, in this case PennyLane’s
standard qubit simulator:

In [4]:
dev = qml.device("default.qubit", wires=qubits)

Next, we need to define the quantum circuit that prepares the trial state of the
molecule. We want to prepare states of the form,

$$
    \vert \Psi(\theta) \rangle = \cos(\theta/2)~|1100\rangle -\sin(\theta/2)~|0011\rangle,
$$

where $\theta$ is the variational parameter to be optimized in order to find
the best approximation to the true ground state. In the Jordan-Wigner encoding,
the first term $|1100\rangle$ represents the [Hartree-Fock (HF) state](https://en.wikipedia.org/wiki/Hartree%E2%80%93Fock_method>)
where the two electrons in
the molecule occupy the lowest-energy orbitals. The second term $|0011\rangle$
encodes a double excitation of the HF state where the two particles are excited from
qubits 0, 1 to 2, 3.

The quantum circuit to prepare the trial state $\vert \Psi(\theta) \rangle$ is
schematically illustrated in the figure below.


In this figure, the gate $G^{(2)}$ corresponds to the
`pennylane.DoubleExcitation` operation, implemented in PennyLane
as a [Givens rotation](https://en.wikipedia.org/wiki/Givens_rotation>), which couples
the four-qubit states $\vert 1100 \rangle$ and $\vert 0011 \rangle$.

Implementing the circuit above using PennyLane is straightforward. First, we use the
`hf_state` function to generate the vector representing the Hartree-Fock state.

In [5]:
electrons = 2
hf = qml.qchem.hf_state(electrons, qubits)
print(hf)

[1 1 0 0]


The `hf` array is used by the `pennylane.BasisState` operation to initialize
the qubit register. Then, we just act with the `pennylane.DoubleExcitation` operation
on the four qubits. The next step is to compute the expectation value
of the molecular Hamiltonian in the trial state prepared by the circuit.
We do this using the `expval` function. The decorator syntax allows us to
run the cost function as an executable QNode with the gate parameter $\theta$:

In [6]:
@qml.qnode(dev)
def circuit(param, wires):
    qml.BasisState(hf, wires=wires)
    qml.DoubleExcitation(param, wires=[0, 1, 2, 3])
    return qml.expval(H)

We can now define our error function simply as the expected value calculated above:

In [7]:
def cost_fn(param):
    return circuit(param, wires=range(qubits))

We initialize the circuit parameter $\theta$ to zero, meaning that we start
from the Hartree-Fock state.

In [8]:
theta = np.array(np.pi / 2)

Here we use a basic gradient-descent optimizer.
We carry out the optimization over a maximum of 100 steps aiming to reach a
convergence tolerance of $10^{-6}$ for the value of the cost function.

In [9]:
max_iterations = 5
convergence_tolerance = 1e-06
optimizer = qml.GradientDescentOptimizer(stepsize=0.01)

And we run the algorithm!

In [10]:
for _ in range(max_iterations):
    theta, prev_energy = optimizer.step_and_cost(cost_fn, theta)
    energy = cost_fn(theta)
    if np.abs(energy - prev_energy) <= convergence_tolerance:
        break

print(f"Final value of the ground-state energy = {energy:.8f} Ha")
print(f"Optimal value of the circuit parameter = {theta:.4f}")

Final value of the ground-state energy = -0.54146578 Ha
Optimal value of the circuit parameter = 1.5316


To submit our results to **aqora** we need to set the output variable in our notebook

In [ ]:
output = float(energy)

-0.5414657821352816


Congratulations!!!